In [1]:
import pandas as pd

In [2]:
# read json with first rows and convert to df
train_df = pd.read_json('../data/bird-sql/train/train.json')
dev = pd.read_json('../data/bird-sql/dev/dev.json')

In [3]:
print(train_df.shape)
print(dev.shape)

(9428, 4)
(1534, 6)


In [13]:
9428+1534

10962

In [4]:
#leave only records where evidence is not empty
train_df = train_df[train_df['evidence'].apply(lambda x: len(x) > 0)]
dev = dev[dev['evidence'].apply(lambda x: len(x) > 0)]
# add a column with nr of records id as strings
train_df['question_id'] = train_df.index.astype(str)
dev['question_id'] = dev['question_id'].astype(str)
# drop column 'difficulty'
dev.drop(columns='difficulty', inplace=True)

# reorder columns for train_df
train_df = train_df[['question_id', 'db_id', 'question', 'evidence', 'SQL']]

In [5]:
print(train_df.shape)
print(dev.shape)

(8783, 5)
(1380, 5)


In [6]:
# split train_df into train and val stratified by label: db_id
from sklearn.model_selection import train_test_split
train_tr, train_val = train_test_split(train_df, test_size=0.4, stratify=train_df['db_id'], random_state=42)

In [7]:
print(train_tr.shape)
print(train_val.shape)
print(dev.shape)

(5269, 5)
(3514, 5)
(1380, 5)


In [11]:
print(train_tr.db_id.nunique())
print(train_val.db_id.nunique())
print(dev.db_id.nunique())

69
69
11


In [8]:
# add label
train_tr['label'] = 'train_tr'
train_val['label'] = 'train_val'
dev['label'] = 'dev'

# union all dataframes
data = pd.concat([train_tr, train_val, dev])

In [12]:
data.shape

(10163, 6)

In [91]:
data.to_json('../data/preprocessed/combined_data.json', orient='records', lines=False, indent=4)